## XỬ LÝ DÒNG TRÙNG LẶP VÀ OUTLIER

In [1]:
# IMPORT CÁC THƯ VIỆN CẦN THIẾT
import pandas as pd
import numpy as np
from scipy.stats import zscore

In [2]:
# ĐỌC DỮ LIỆU
path = 'Merge_Data.csv'
raw_df = pd.read_csv(path, sep=',', encoding='utf-8')
# Hiển thị mẫu
raw_df.head(5)

,timestamp,Temprature (Kelvin),Feels like (Kelvin),Pressure (mb),Humidity (%),Min Temprature (Kelvin),Max Temprature (Kelvin),Wind speed (m/s),Wind degree,Wind gust (m/s),...,Weather details,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,1701388800,301.16,306.75,1011,89,301.16,301.16,0.51,0,0.0,...,"id: 802, main: Clouds, description: scattered ...",5,5661.01,128.75,42.50,0.0,56.74,246.02,294.09,29.64
1,1701392400,301.16,306.75,1011,89,301.16,301.16,0.51,300,0.0,...,"id: 802, main: Clouds, description: scattered ...",5,4699.71,109.08,34.62,0.0,42.44,199.75,233.37,22.29
2,1701396000,301.16,306.75,1010,89,301.16,301.16,0.51,0,0.0,...,"id: 802, main: Clouds, description: scattered ...",5,3791.81,84.94,26.73,0.0,29.80,157.48,177.82,17.23
3,1701399600,300.16,304.35,1009,94,300.16,300.16,0.51,330,0.0,...,"id: 802, main: Clouds, description: scattered ...",5,3017.43,61.69,20.91,0.0,22.41,121.27,132.87,13.05
4,1701403200,299.16,299.16,1009,94,299.16,299.16,0.51,290,0.0,...,"id: 701, main: Mist, description: mist, icon: 50n",5,2456.67,46.04,17.82,0.0,19.55,96.76,104.39,10.26


### Xử lý dòng trùng lặp

In [3]:
# Loại bỏ các dòng trùng lặp
raw_df.drop_duplicates(keep='first', inplace=True)

### Xử lý outlier

Phương pháp phát hiện outlier:  Sử dụng $Z_{score}$ để phát hiện outlier, dữ liệu có giá trị $Z_{score}$ > 3 được coi là outlier.<br>
Phương pháp xử lý outlier: Sử dụng giá trị median để thay thế cho outlier. 

Giải thích lý do sử dụng giá trị median: 
- Duy trì kích thước dữ liệu, tránh mất mát so với việc loại bỏ các dòng outlier
- Median giúp giảm ảnh hưởng đến bộ dữ liệu, đặc biệt là phân phối của dữ liệu

In [4]:
def detect_and_replace_outliers_zscore(series, threshold=3):
    """Detect outliers using Z-score and replace them with the median."""
    # Tính toán Z-score cho series
    z_scores = zscore(series)
    
    # Tính toán median
    median_value = series.median()
    
    # Tạo một mask cho các outliers
    mask = np.abs(z_scores) > threshold
    
    # Trả về một series đã được thay đổi
    return pd.Series(np.where(mask, median_value, series))

In [5]:
# Chọn ra các cột cần xử lý outlier
numeric_columns = raw_df.select_dtypes(include='number').columns.difference(['timestamp'])

In [6]:
# Xử lý
for col in numeric_columns:
    raw_df[col] = detect_and_replace_outliers_zscore(raw_df[col])

### Lưu dữ liệu đã xử lý

In [7]:
raw_df.to_csv('cleaned_data.csv', index=False)